In [3]:
import psycopg2
from faker import Faker
import random

conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="obama")



In [4]:
from datetime import datetime, timedelta

def generate_unique_dates(end_date, num_dates):
    date_set = set()
    dates = []
    while len(dates) < num_dates:
        current_date = fake.date_between_dates(datetime.now().date()-timedelta(days=30*365), end_date)
        if current_date not in date_set:
            date_set.add(current_date)
            dates.append(current_date)
    return dates

def generate_random_number_of_items(items, max_count):
    count = random.randint(1, max_count)
    return random.sample(items, min(count, len(items)))

In [5]:
fake = Faker()

numSites = 5
numFormats = 10
numSessions = 10000
numTournaments = 100
formatsPerTournament = 2
tournamentsPerSession = 5
numTransactions = 10

si_sites_data = [(fake.word(),) for _ in range(numSites)]
f_formats_data = [(fake.word(),) for _ in range(numFormats)]

current_date = datetime.now().date()
s_sessions_data = [(date, random.randint(1, 14)) for date in generate_unique_dates(current_date, numSessions)]

t_tournaments_data = [(fake.word(), random.randint(1, 100), random.randint(0, 10000), random.randint(1, numSites)) for _ in range(numTournaments)]

formats_shuffled = random.sample(range(1, numFormats + 1), numFormats)

tournament_formats_mapping = {x+1: generate_random_number_of_items(formats_shuffled, formatsPerTournament) for x in range(numTournaments)}

tournaments_shuffled = random.sample(range(1, numTournaments + 1), numTournaments)
session_tournaments_mapping = {x+1: generate_random_number_of_items(tournaments_shuffled, tournamentsPerSession) for x in range(numSessions)}

tr_transactions_data = [(random.randint(1, numSites), random.randint(-1000, 1000), date) for date in generate_unique_dates(current_date, numTransactions)]

cur = conn.cursor()

for i in si_sites_data:
    sql = """INSERT INTO si_sites(si_name) VALUES(%s)"""
    cur.execute(sql, i)
conn.commit()

for i in f_formats_data:
    sql = """INSERT INTO f_formats(f_name)
             VALUES(%s);"""
    cur.execute(sql, i)
conn.commit()

for i in s_sessions_data:
    sql = """INSERT INTO s_sessions(s_date, s_length)
             VALUES(%s, %s);"""
    cur.execute(sql, i)
conn.commit()

for i in t_tournaments_data:
    sql = """INSERT INTO t_tournaments(t_name, t_buyIn, t_guarantee, t_si_site)
             VALUES(%s, %s, %s, %s);"""
    cur.execute(sql, i)
conn.commit()

for tournament, formats in tournament_formats_mapping.items():
    for f_id in formats:
        sql = """INSERT INTO hf_hasFormat(hf_f_id, hf_t_id)
                 VALUES(%s, %s);"""
        cur.execute(sql, (f_id, tournament))
conn.commit()

for session, tournaments in session_tournaments_mapping.items():
    for t_id in tournaments:
        sql = """INSERT INTO ht_hasTournament(ht_s_id, ht_t_id, ht_winnings, ht_position)
                 VALUES(%s, %s, %s, %s);"""
        cur.execute(sql, (session, t_id, random.randint(0, 10000), random.randint(1, 500)))
conn.commit()

for i in tr_transactions_data:
    sql = """INSERT INTO tr_transactions(tr_si_id, tr_amount, tr_date)
             VALUES(%s, %s, %s);"""
    cur.execute(sql, i)
conn.commit()

print("Done")

Done
